In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
diabetes_df = pd.read_csv('../indicators_bmi_filtered.csv')
diabetes_df.head()

,DIABETE4,_RACE,TOLDHI3,BPHIGH6,_BMI5,SMOKE100,_RFBING5,EDUCA,GENHLTH,_AGEG5YR,EXERANY2,FRUIT2,VEGETAB2,_INCOMG1,MEDCOST1,_SEX
0,No,White,Yes,No,14.54,Yes,Yes,High School Grad,Poor,Age 70 to 74,No,Yes,Yes,"$25,000 to < $35,000",No,Female
1,Yes,Black,No,Yes,28.29,No,Yes,High School Grad,Very Good,Age 70 to 74,No,Yes,Yes,"$15,000 to < $25,000",No,Female
2,Yes,White,Yes,Yes,33.47,No,No,High School Grad,Very Good,Age 60 to 64,Yes,Yes,Yes,"$50,000 to < $100,000",No,Female
3,Yes,Multiracial,Yes,No,28.73,No,Yes,Some High School,Poor,Age 75 to 79,Yes,Yes,Yes,"$15,000 to < $25,000",No,Male
4,No,White,No,No,24.37,Yes,Yes,Some College,Good,Age 80 or older,No,Yes,Yes,"$35,000 to < $50,000",No,Male


In [3]:
diabetes_df.dtypes

DIABETE4     object
_RACE        object
TOLDHI3      object
BPHIGH6      object
_BMI5       float64
SMOKE100     object
_RFBING5     object
EDUCA        object
GENHLTH      object
_AGEG5YR     object
EXERANY2     object
FRUIT2       object
VEGETAB2     object
_INCOMG1     object
MEDCOST1     object
_SEX         object
dtype: object

In [4]:
# Generate our categorical variable list
indicators_cat = diabetes_df.dtypes[diabetes_df.dtypes == "object"].index.tolist()

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(diabetes_df[indicators_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names_out(indicators_cat)
encode_df.head()

,DIABETE4_No,DIABETE4_Yes,_RACE_ Native Hawaiian or other Pacific Islander,_RACE_American Indian or Alaskan Native,_RACE_Asian,_RACE_Black,_RACE_Hispanic,_RACE_Multiracial,_RACE_Other,_RACE_White,...,"_INCOMG1_$100,000 to < $200,000","_INCOMG1_$15,000 to < $25,000","_INCOMG1_$25,000 to < $35,000","_INCOMG1_$35,000 to < $50,000","_INCOMG1_$50,000 to < $100,000","_INCOMG1_Less than $15,000",MEDCOST1_No,MEDCOST1_Yes,_SEX_Female,_SEX_Male
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [6]:
# Merge one-hot encoded features and drop the originals
diabetes_df = diabetes_df.merge(encode_df,left_index=True, right_index=True)
diabetes_df = diabetes_df.drop(indicators_cat,1)
diabetes_df.head()

C:\Users\ashle\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,_BMI5,DIABETE4_No,DIABETE4_Yes,_RACE_ Native Hawaiian or other Pacific Islander,_RACE_American Indian or Alaskan Native,_RACE_Asian,_RACE_Black,_RACE_Hispanic,_RACE_Multiracial,_RACE_Other,...,"_INCOMG1_$100,000 to < $200,000","_INCOMG1_$15,000 to < $25,000","_INCOMG1_$25,000 to < $35,000","_INCOMG1_$35,000 to < $50,000","_INCOMG1_$50,000 to < $100,000","_INCOMG1_Less than $15,000",MEDCOST1_No,MEDCOST1_Yes,_SEX_Female,_SEX_Male
0,14.54,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,28.29,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,33.47,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,28.73,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,24.37,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [7]:
# Define features and targets
y = diabetes_df['DIABETE4_Yes']
X = diabetes_df.drop(columns=['DIABETE4_Yes', 'DIABETE4_No'])

In [8]:
# Split into training and testing 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 78, stratify=y)

In [9]:
# Create Standard Scaler
scaler = StandardScaler()

# Fit Data
X_scaler = scaler.fit(X_train)

# Scale Data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Create and logistic regression model 
classifier = LogisticRegression(random_state=78)
classifier

LogisticRegression(random_state=78)

In [11]:
# Fit and train model 
clf = classifier.fit(X_train_scaled, y_train)

In [12]:
predictions = classifier.predict(X_test_scaled)

In [13]:
# Calculate accuracy score 
accuracy_score(y_test, predictions)

0.8559990774451821

In [14]:
# Generate confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create df from confusion matrix
cm_df = pd.DataFrame(cm, index=['Actual 0', 'Acutal 1'], columns = ['Predicted 0', 'Predicted 1'])
cm_df

,Predicted 0,Predicted 1
Actual 0,50910,834
Acutal 1,7907,1050


In [15]:
# classification report
report = classification_report(y_test, predictions)
print(report)

              precision    recall  f1-score   support

         0.0       0.87      0.98      0.92     51744
         1.0       0.56      0.12      0.19      8957

    accuracy                           0.86     60701
   macro avg       0.71      0.55      0.56     60701
weighted avg       0.82      0.86      0.81     60701



In [16]:
print(clf.coef_)

[[ 0.35783141  0.02302104  0.05248545  0.06039671  0.03930312  0.04057779
  -0.00747346 -0.00872652 -0.08798864 -0.12296389  0.12296389 -0.17022664
   0.17022664  0.00794488 -0.00794488 -0.09659882  0.09659882 -0.01866784
   0.00485244 -0.00681462  0.00922327  0.02249885  0.00765625 -0.29850018
   0.24110669  0.14110828  0.16848508 -0.12732874 -0.25782703 -0.2073085
  -0.18675829 -0.14393051 -0.07654453 -0.02763794  0.01754433  0.06818347
   0.07712547  0.11766098  0.14603431  0.12854761  0.07753226  0.03817384
  -0.03817384 -0.00506968  0.00506968  0.0014955  -0.0014955  -0.05816891
  -0.0386362   0.03677747  0.03472952  0.01625446 -0.0115078   0.03407499
   0.00285849 -0.00285849 -0.03928484  0.03928484]]


In [17]:
coef = pd.DataFrame(zip(X_train, np.transpose(clf.coef_.tolist()[0])), columns=['features', 'coef'])
coef

,features,coef
0,_BMI5,0.357831
1,_RACE_ Native Hawaiian or other Pacific Islander,0.023021
2,_RACE_American Indian or Alaskan Native,0.052485
3,_RACE_Asian,0.060397
4,_RACE_Black,0.039303
5,_RACE_Hispanic,0.040578
6,_RACE_Multiracial,-0.007473
7,_RACE_Other,-0.008727
8,_RACE_White,-0.087989
9,TOLDHI3_No,-0.122964


In [18]:
coef.sort_values('coef')

,features,coef
23,GENHLTH_Excellent,-0.298500
28,_AGEG5YR_Age 18 to 24,-0.257827
29,_AGEG5YR_Age 25 to 29,-0.207308
30,_AGEG5YR_Age 30 to 34,-0.186758
11,BPHIGH6_No,-0.170227
31,_AGEG5YR_Age 35 to 39,-0.143931
27,GENHLTH_Very Good,-0.127329
9,TOLDHI3_No,-0.122964
15,_RFBING5_No,-0.096599
8,_RACE_White,-0.087989
